In [1]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import pickle

C:\Users\aenea\Desktop\Studium\PP2\protfrag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dropo

In [4]:
# usage
import re
sequence_examples = ["PRTEINO", "SEQWENCE"]
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]
sequence_examples

['P R T E I N X', 'S E Q W E N C E']

In [5]:
ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")
ids

{'input_ids': [[13, 8, 11, 9, 12, 17, 23, 1, 0], [7, 9, 16, 21, 9, 17, 22, 9, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [6]:
input_ids = torch.tensor(ids['input_ids']).to(device)
input_ids

tensor([[13,  8, 11,  9, 12, 17, 23,  1,  0],
        [ 7,  9, 16, 21,  9, 17, 22,  9,  1]], device='cuda:0')

In [7]:
attention_mask = torch.tensor(ids['attention_mask']).to(device)
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [8]:
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)
embedding_repr

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 0.1172, -0.0473,  0.0297,  ...,  0.0785,  0.0417,  0.1689],
         [ 0.2413, -0.1148, -0.2084,  ...,  0.0521, -0.1574,  0.0938],
         [ 0.4049, -0.1518,  0.0273,  ...,  0.0850, -0.0266,  0.0269],
         ...,
         [ 0.3105, -0.0559, -0.2307,  ...,  0.0486,  0.3549, -0.1278],
         [-0.1563, -0.0111,  0.2014,  ..., -0.0067,  0.0096, -0.0554],
         [-0.3005,  0.0338,  0.1206,  ...,  0.1091, -0.1837,  0.0503]],

        [[ 0.1114, -0.1654, -0.2758,  ...,  0.0380,  0.1607, -0.0633],
         [ 0.2302, -0.1616, -0.1128,  ...,  0.0071, -0.0874,  0.0404],
         [ 0.2764,  0.0426, -0.2498,  ...,  0.2141, -0.0559,  0.0231],
         ...,
         [ 0.0186, -0.0554,  0.1317,  ..., -0.1865,  0.1502, -0.1360],
         [ 0.1116, -0.0441,  0.2025,  ..., -0.0112, -0.1201, -0.0566],
         [-0.0797, -0.0545,  0.1329,  ...,  0.0111, -0.0254,  0.0034]]],
       device='cuda:0'), past_key_values=None, hidden_st

In [9]:
emb_0 = embedding_repr.last_hidden_state[0,:7] # shape (7 x 1024)
print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")

Shape of per-residue embedding of first sequences: torch.Size([7, 1024])


In [10]:
emb_1 = embedding_repr.last_hidden_state[1,:8] # shape (8 x 1024)

In [11]:
emb_0

tensor([[ 0.1172, -0.0473,  0.0297,  ...,  0.0785,  0.0417,  0.1689],
        [ 0.2413, -0.1148, -0.2084,  ...,  0.0521, -0.1574,  0.0938],
        [ 0.4049, -0.1518,  0.0273,  ...,  0.0850, -0.0266,  0.0269],
        ...,
        [ 0.2883, -0.1204, -0.0958,  ...,  0.1133, -0.1676,  0.1846],
        [-0.0326, -0.0874, -0.1193,  ..., -0.0963, -0.1092, -0.2432],
        [ 0.3105, -0.0559, -0.2307,  ...,  0.0486,  0.3549, -0.1278]],
       device='cuda:0')

In [12]:
emb_0_prot = emb_0.mean(dim=0)
emb_0_prot

tensor([ 0.2284, -0.1131, -0.0947,  ...,  0.0571, -0.0393,  0.0195],
       device='cuda:0')

In [13]:
emb_0_prot.shape

torch.Size([1024])

In [4]:
import pandas as pd
import torch
train_df = pd.read_csv("../data/custom_fragments2/datasets/random_equal_distribution/random_equal_train.csv")

In [5]:
train_df

,source_accession,fragment_type,sequence,is_fragment,fragment_length,source_length
0,A3Q8S8,mixed,MNGEKVKRLSTLAETLPIQVITPESFSLLFEGPKSRRQFIDWGAFH...,True,233,360
1,Q6GLS4,terminal_N,MPFGCVTLGDKKDYNHPTEVSDRYDLGQLIKTEEFCEVFRAKEKSS...,True,191,377
2,P02607,complete,MCDFSEEQTAEFKEAFQLFDRTGDGKILYSQCGDVMRALGQNPTNA...,False,151,151
3,Q4WRR0,complete,MGLTSILIAQVLFLGAANSAVVKRWPCSVSPTGPTDPSVAKNCGYW...,False,299,299
4,A9INS6,complete,MELDIKAIMERLPHRYPMLLVDRVLDIVPGKSIVAIKNVSINEPFF...,False,151,151
...,...,...,...,...,...,...
478595,E2RYF7,terminal_both,SISTKHPETSPKDSRIRENDVTADGRTTEDHITADPGTTEDSVTAD...,True,110,251
478596,A8GLX9,terminal_C,NSLNRKHYSLSFDHQNNFSNLQERARAGRYDLMTNLCLEFDILVLL...,True,356,430
478597,P43934,terminal_C,IAGAIQKFEFVYELSLKMMKRQLQQDAINTDDIGAYGFKDILREAL...,True,104,146
478598,Q8TX15,complete,MRAPEGFLLGGIKREGIGVGLIFSERRCAVAGTFTENTLRAAPVEH...,False,387,387


In [6]:
len(train_df)

478600

In [7]:
length_filtered_df = train_df[train_df["fragment_length"] <= 512]
length_filtered_df

,source_accession,fragment_type,sequence,is_fragment,fragment_length,source_length
0,A3Q8S8,mixed,MNGEKVKRLSTLAETLPIQVITPESFSLLFEGPKSRRQFIDWGAFH...,True,233,360
1,Q6GLS4,terminal_N,MPFGCVTLGDKKDYNHPTEVSDRYDLGQLIKTEEFCEVFRAKEKSS...,True,191,377
2,P02607,complete,MCDFSEEQTAEFKEAFQLFDRTGDGKILYSQCGDVMRALGQNPTNA...,False,151,151
3,Q4WRR0,complete,MGLTSILIAQVLFLGAANSAVVKRWPCSVSPTGPTDPSVAKNCGYW...,False,299,299
4,A9INS6,complete,MELDIKAIMERLPHRYPMLLVDRVLDIVPGKSIVAIKNVSINEPFF...,False,151,151
...,...,...,...,...,...,...
478595,E2RYF7,terminal_both,SISTKHPETSPKDSRIRENDVTADGRTTEDHITADPGTTEDSVTAD...,True,110,251
478596,A8GLX9,terminal_C,NSLNRKHYSLSFDHQNNFSNLQERARAGRYDLMTNLCLEFDILVLL...,True,356,430
478597,P43934,terminal_C,IAGAIQKFEFVYELSLKMMKRQLQQDAINTDDIGAYGFKDILREAL...,True,104,146
478598,Q8TX15,complete,MRAPEGFLLGGIKREGIGVGLIFSERRCAVAGTFTENTLRAAPVEH...,False,387,387


In [8]:
# generate embeddings for all
import re

length_filtered_df["sequence_split"] = length_filtered_df["sequence"].apply(
    lambda seq: " ".join(list(re.sub(r"[UZOB]", "X", seq)))
)

C:\Users\aenea\AppData\Local\Temp\ipykernel_4500\771708216.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  length_filtered_df["sequence_split"] = length_filtered_df["sequence"].apply(


In [9]:
length_filtered_df

,source_accession,fragment_type,sequence,is_fragment,fragment_length,source_length,sequence_split
0,A3Q8S8,mixed,MNGEKVKRLSTLAETLPIQVITPESFSLLFEGPKSRRQFIDWGAFH...,True,233,360,M N G E K V K R L S T L A E T L P I Q V I T P ...
1,Q6GLS4,terminal_N,MPFGCVTLGDKKDYNHPTEVSDRYDLGQLIKTEEFCEVFRAKEKSS...,True,191,377,M P F G C V T L G D K K D Y N H P T E V S D R ...
2,P02607,complete,MCDFSEEQTAEFKEAFQLFDRTGDGKILYSQCGDVMRALGQNPTNA...,False,151,151,M C D F S E E Q T A E F K E A F Q L F D R T G ...
3,Q4WRR0,complete,MGLTSILIAQVLFLGAANSAVVKRWPCSVSPTGPTDPSVAKNCGYW...,False,299,299,M G L T S I L I A Q V L F L G A A N S A V V K ...
4,A9INS6,complete,MELDIKAIMERLPHRYPMLLVDRVLDIVPGKSIVAIKNVSINEPFF...,False,151,151,M E L D I K A I M E R L P H R Y P M L L V D R ...
...,...,...,...,...,...,...,...
478595,E2RYF7,terminal_both,SISTKHPETSPKDSRIRENDVTADGRTTEDHITADPGTTEDSVTAD...,True,110,251,S I S T K H P E T S P K D S R I R E N D V T A ...
478596,A8GLX9,terminal_C,NSLNRKHYSLSFDHQNNFSNLQERARAGRYDLMTNLCLEFDILVLL...,True,356,430,N S L N R K H Y S L S F D H Q N N F S N L Q E ...
478597,P43934,terminal_C,IAGAIQKFEFVYELSLKMMKRQLQQDAINTDDIGAYGFKDILREAL...,True,104,146,I A G A I Q K F E F V Y E L S L K M M K R Q L ...
478598,Q8TX15,complete,MRAPEGFLLGGIKREGIGVGLIFSERRCAVAGTFTENTLRAAPVEH...,False,387,387,M R A P E G F L L G G I K R E G I G V G L I F ...


In [10]:
input_df = pd.DataFrame(data=length_filtered_df[["sequence_split", "is_fragment"]])
input_df

,sequence_split,is_fragment
0,M N G E K V K R L S T L A E T L P I Q V I T P ...,True
1,M P F G C V T L G D K K D Y N H P T E V S D R ...,True
2,M C D F S E E Q T A E F K E A F Q L F D R T G ...,False
3,M G L T S I L I A Q V L F L G A A N S A V V K ...,False
4,M E L D I K A I M E R L P H R Y P M L L V D R ...,False
...,...,...
478595,S I S T K H P E T S P K D S R I R E N D V T A ...,True
478596,N S L N R K H Y S L S F D H Q N N F S N L Q E ...,True
478597,I A G A I Q K F E F V Y E L S L K M M K R Q L ...,True
478598,M R A P E G F L L G G I K R E G I G V G L I F ...,False


In [11]:
# actual embedding generation
split_seqs = list(input_df["sequence_split"])
fragment_data = list(input_df["is_fragment"])

In [12]:
len(split_seqs)

407114

In [13]:
from tqdm import tqdm

In [14]:
batch_size = 16
input_ids_list = []
attention_mask_list = []

In [15]:
for i in tqdm(range(0, len(split_seqs), batch_size)):
    batch = split_seqs[i:i+batch_size]

    encoded = tokenizer.batch_encode_plus(
        batch,
        add_special_tokens=True,
        padding="longest",
        return_tensors="pt"
    )

    input_ids_list.append(encoded["input_ids"])
    attention_mask_list.append(encoded["attention_mask"])

100%|██████████| 25445/25445 [01:35<00:00, 267.52it/s]


In [16]:
input_ids_list[0]

tensor([[19, 17,  5,  ...,  0,  0,  0],
        [19, 13, 15,  ...,  0,  0,  0],
        [19, 22, 10,  ...,  0,  0,  0],
        ...,
        [ 8,  5,  5,  ...,  0,  0,  0],
        [19,  7, 14,  ...,  0,  0,  0],
        [19,  9,  8,  ...,  0,  0,  0]])

In [17]:
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids_list[0].to(device), attention_mask=attention_mask_list[0].to(device))

embedding_repr.last_hidden_state.shape

torch.Size([16, 384, 1024])

In [28]:
all_embeddings = []

In [29]:
# load existing embeddings
with open("./embeddings/embeddings_5000.pkl", "rb") as f:
    all_embeddings = pickle.load(f)

all_embeddings

[tensor([[ 0.0252, -0.0285,  0.0164,  ...,  0.0135,  0.0188,  0.0185],
         [-0.0035,  0.0681,  0.0327,  ..., -0.0282,  0.0312,  0.0188],
         [ 0.0359,  0.0725,  0.0162,  ..., -0.0769, -0.0088,  0.0452],
         ...,
         [-0.0153,  0.0184,  0.0302,  ...,  0.0097, -0.0302, -0.0015],
         [ 0.0322,  0.0244,  0.0176,  ...,  0.0388,  0.0485,  0.0391],
         [ 0.0118,  0.0336,  0.0571,  ..., -0.0237, -0.0143,  0.0197]]),
 tensor([[ 0.0252, -0.0285,  0.0164,  ...,  0.0135,  0.0188,  0.0185],
         [-0.0035,  0.0681,  0.0327,  ..., -0.0282,  0.0312,  0.0188],
         [ 0.0359,  0.0725,  0.0162,  ..., -0.0769, -0.0088,  0.0452],
         ...,
         [-0.0153,  0.0184,  0.0302,  ...,  0.0097, -0.0302, -0.0015],
         [ 0.0322,  0.0244,  0.0176,  ...,  0.0388,  0.0485,  0.0391],
         [ 0.0118,  0.0336,  0.0571,  ..., -0.0237, -0.0143,  0.0197]]),
 tensor([[-0.0335,  0.0176,  0.0200,  ..., -0.0383, -0.0208,  0.0091],
         [ 0.0078, -0.0029,  0.0483,  ..., -0

In [30]:
len(all_embeddings)

5005

In [31]:
with torch.no_grad():
    for (i, (input_ids, attention_masks)) in tqdm(enumerate(zip(input_ids_list, attention_mask_list)), total=len(input_ids_list)):
        if i < 5000:
            continue

        embeddings = model(input_ids=input_ids.to(device), attention_mask=attention_masks.to(device)).last_hidden_state

        mask = attention_masks.unsqueeze(-1).expand(embeddings.size()).float().to(device)

        summed = torch.sum(embeddings * mask, dim=1)
        lengths = torch.sum(mask, dim=1)
        prot_embed = summed / lengths
        all_embeddings.append(prot_embed.cpu())

        if i % 10 == 0:
            print(i)

        if i % 5000 == 0:
            with open(f"./embeddings/embeddings_{i}.pkl", "wb") as f:
                pickle.dump(all_embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)


with open(f"./embeddings/final_embeddings.pkl", "wb") as f:
    pickle.dump(all_embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)

all_embeddings = torch.cat(all_embeddings, dim=0)

  0%|          | 0/25445 [00:00<?, ?it/s]

5000


 20%|█▉        | 5001/25445 [00:01<00:06, 3197.00it/s]

5010


 20%|█▉        | 5020/25445 [00:18<01:48, 188.10it/s] 

...

100%|█████████▉| 25431/25445 [5:22:06<00:12,  1.08it/s]

25430


100%|█████████▉| 25441/25445 [5:22:15<00:03,  1.08it/s]

25440


100%|██████████| 25445/25445 [5:22:18<00:00,  1.32it/s]


In [32]:
all_embeddings.shape

torch.Size([407194, 1024])

In [33]:
# check against duplicates
all_embeddings.unique(dim=0).shape

torch.Size([406618, 1024])

In [34]:
# save final tensor
with open("./embeddings/final_embeddings_tensor.pkl", "wb") as p:
    pickle.dump(all_embeddings.unique(dim=0), p)

In [36]:
# write to csv
all_embeddings[0]

tensor([ 0.0252, -0.0285,  0.0164,  ...,  0.0135,  0.0188,  0.0185])

In [40]:
all_embeddings.unique(dim=0, sorted=False)[0]

tensor([-0.2020,  0.0967,  0.1084,  ...,  0.0571,  0.1519,  0.0648])

In [38]:
with open("./embeddings/final_embeddings.pkl", "rb") as p:
    embeddings_list = pickle.load(p)

In [39]:
embeddings_list[0][0]

tensor([ 0.0252, -0.0285,  0.0164,  ...,  0.0135,  0.0188,  0.0185])